In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
import zipfile
import os
from pyspark.sql import DataFrame
from pyspark.sql.functions import current_timestamp

In [0]:
%run Config/storage_config

In [0]:
%run Config/secrets_config

In [0]:
input_path = f"{cnpj_path}"

In [0]:
dbutils.fs.ls(input_path)

In [0]:
# Deszipando
input_path = f"{cnpj_path}"  
# TODO Mover isso para um local adequado
output_path = "/dbfs/tmp/deszipados"  # pasta de destino (DBFS)

# Cria a pasta de destino se não existir
os.makedirs(output_path, exist_ok=True)

# Lista arquivos na pasta do DBFS
arquivos = dbutils.fs.ls(input_path)


def decompress_file(arquivo):
    """Descompassa um arquivo zip no DBFS.
    """
    if arquivo.name.endswith(".zip"):
        # Caminho do zip no DBFS
        caminho_zip_dbfs = arquivo.path

        # Caminho local temporário para descompactar
        caminho_zip_local = f"/tmp/{arquivo.name}"

        # Copia do DBFS para local
        dbutils.fs.cp(caminho_zip_dbfs, f"file:{caminho_zip_local}")

        # Cria uma subpasta para cada zip
        nome_subpasta = os.path.splitext(arquivo.name)[0]
        caminho_subpasta = os.path.join(output_path, nome_subpasta)
        os.makedirs(caminho_subpasta, exist_ok=True)

        # Descompacta
        with zipfile.ZipFile(caminho_zip_local, 'r') as zip_ref:
            zip_ref.extractall(caminho_subpasta)
        
        print(f"Descompactado {arquivo.name} em {caminho_subpasta}")

# Descomprime cada arquivo da lista 'arquivos'
[decompress_file(arquivo) for arquivo in arquivos]

# for arquivo in arquivos:
#     if arquivo.name.endswith(".zip"):
#         # Caminho do zip no DBFS
#         caminho_zip_dbfs = arquivo.path

#         # Caminho local temporário para descompactar
#         caminho_zip_local = f"/tmp/{arquivo.name}"

#         # Copia do DBFS para local
#         dbutils.fs.cp(caminho_zip_dbfs, f"file:{caminho_zip_local}")

#         # Cria uma subpasta para cada zip
#         nome_subpasta = os.path.splitext(arquivo.name)[0]
#         caminho_subpasta = os.path.join(output_path, nome_subpasta)
#         os.makedirs(caminho_subpasta, exist_ok=True)

#         # Descompacta
#         with zipfile.ZipFile(caminho_zip_local, 'r') as zip_ref:
#             zip_ref.extractall(caminho_subpasta)
        
#         print(f"Descompactado {arquivo.name} em {caminho_subpasta}")


In [0]:
t# Listando os arquivos deszipados
display(dbutils.fs.ls(cnpj_path))

In [0]:
# Arquivos de empresas
# Colunas 
# Coluna dos arquivos de EMPRESAS
# TODO colocar a definição do esquema em um lugar mais apropriado a definir.
empresas_columns = [
    "CNPJ_BASICO", 
    "RAZAO_SOCIAL_NOME_EMPRESARIAL", 
    "NATUREZA_JURIDICA", 
    "QUALIFICACAO_DO_RESPONSAVEL",  
    "CAPITAL_SOCIAL_DA_EMPRESA",
    "PORTE_DA_EMPRESA",
    "ENTE_FEDERATIVO_RESPONSAVEL"
]

# Lendo e unificando os arquivos Empresas0,1,2,3...
# cnpj_path = f"abfss://cnpj@{storage_input}.dfs.core.windows.net"
# empresas_df = spark.read.csv(
#     "dbfs:/tmp/deszipados/Empresas*/*.EMPRECSV", 
#     header=False, 
#     encoding="iso-8859-1", sep=';'
# )

empresas_df = spark.read.csv(
    f"dbfs:/tmp/deszipados/Empresas*/*.EMPRECSV", 
    header=False, 
    encoding="iso-8859-1", sep=';'
)

empresas_df = empresas_df.toDF(*empresas_columns)
# Criando uma coluna para gravar o caminho de origem dos dados
empresas_df = empresas_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Empresas*/*.EMPRECSV"))
# Criando uma coluna para a data de ingestão
empresas_df = empresas_df.withColumn("ingestion_dt", current_timestamp())
print(empresas_df.schema)
display(empresas_df)

print(f"checando o path pre-definido cnpj_path: {cnpj_path}")




In [0]:
# Estabelecimentos
estabelecimentos_columns = [
    "CNPJ_BASICO",
    "CNPJ_ORDEM",
    "CNPJ_DV",
    "IDENTIFICADOR_MATRIZ_FILIAL",
    "NOME_FANTASIA",
    "SITUACAO_CADASTRAL",
    "DATA_SITUACAO_CADASTRAL",
    "MOTIVO_SITUACAO_CADASTRAL",
    "NOME_CIDADE_EXTERIOR",
    "PAIS",
    "DT_INICIO_ATIVIDADE",
    "CNAE_FISCAL_PRINCIPAL",
    "CNAE_FISCAL_SECUNDARIA",
    "TIPO_LOGRADOURO",
    "LOGRADOURO",
    "NUMERO",
    "COMPLEMENTO",
    "BAIRRO",
    "CEP",
    "UF",
    "MUNICIPIO",
    "DDD1",
    "TELEFONE1",
    "DDD2",
    "TELEFONE2",
    "DDD_FAX",
    "FAX",
    "CORREIO_ELETRONICO",
    "SITUACAO_ESPECIAL",
    "DATA_SITUACAO_ESPECIAL"
]
estabelecimentos_df = spark.read.csv("dbfs:/tmp/deszipados/Estabelecimentos*/*.ESTABELE", header=False, encoding="iso-8859-1", sep=';')
estabelecimentos_df = estabelecimentos_df.toDF(*estabelecimentos_columns)

# Criando uma coluna para gravar o caminho de origem dos dados
estabelecimentos_df = estabelecimentos_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Estabelecimentos*/*.ESTABELE"))

estabelecimentos_df = (
    estabelecimentos_df
    .withColumn("ingestion_dt", current_timestamp())
)

display(estabelecimentos_df)


In [0]:
# Arquivos de simples
# Coluna dos arquivos de simples
simples_columns = [
    "CNPJ BÁSICO", 
    "OPCAO PELO SIMPLES", 
    "DATA DE OPCAO PELO SIMPLES", 
    "DATA DE EXCLUSAO DO SIMPLES", 
    "OPCAO PELO MEI", 
    "DATA DE OPCAO PELO MEI", 
    "DATA DE EXCLUSAO DO MEI"
]

# Lendo e unificando os arquivos 
simples_df = spark.read.csv("dbfs:/tmp/deszipados/Simples*", header=False, encoding="iso-8859-1", sep=';')
simples_df = simples_df.toDF(*simples_columns)

# Criando uma coluna para gravar o caminho de origem dos dados
simples_df = simples_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Simples*"))

simples_df = (
    simples_df
    .withColumn("ingestion_dt", current_timestamp())
)

display(simples_df)

In [0]:
# Arquivos de sócios
# Coluna dos arquivos de sócios
socios_columns = [
    "CNPJ BÁSICO", 
    "IDENTIFICADOR DO SÓCIO", 
    "NOME DO SÓCIO", 
    "CPF/CNPJ DO SÓCIO", 
    "QUALIFICACAO DO SÓCIO", 
    "DATA DE ENTRADA NO SOCIEDADE", 
    "PAIS", 
    "REPRESENTANTE LEGAL", 
    "NOME DO REPRESENTANTE LEGAL", 
    "QUALIFICACAO DO REPRESENTANTE LEGAL",
    "FAIXA ETÁRIA"
]

# Lendo e unificando os arquivos 
socios_df = spark.read.csv("dbfs:/tmp/deszipados/Socios*/*.SOCIOCSV", header=False, encoding="iso-8859-1", sep=';')
socios_df = socios_df.toDF(*socios_columns)

# Criando uma coluna para gravar o caminho de origem dos dados
socios_df = socios_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Socios*/*.SOCIOCSV"))

socios_df = (
    socios_df
    .withColumn("ingestion_dt", current_timestamp())
)

display(socios_df)


In [0]:
# Arquivos de Países
# Coluna dos arquivos de Países
paises_columns = [
    "CÓDIGO",
    "DESCRICAO"
]

# Lendo e unificando os arquivos 
paises_df = spark.read.csv("dbfs:/tmp/deszipados/Paises*", header=False, encoding="iso-8859-1", sep=';')
paises_df = paises_df.toDF(*paises_columns)


# Criando uma coluna para gravar o caminho de origem dos dados
paises_df = paises_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Paises*"))

paises_df = (
    paises_df
    .withColumn("ingestion_dt", current_timestamp())
)


display(paises_df)


In [0]:
# Arquivos de Municipios
# Coluna dos arquivos de Municipios
municipios_columns = [
    "CÓDIGO",
    "DESCRICAO"
]

# Lendo e unificando os arquivos 
municipios_df = spark.read.csv("dbfs:/tmp/deszipados/Municipios*", header=False, encoding="iso-8859-1", sep=';')
municipios_df = municipios_df.toDF(*municipios_columns)

# Criando uma coluna para gravar o caminho de origem dos dados
municipios_df = municipios_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Municipios*"))

municipios_df = (
    municipios_df
    .withColumn("ingestion_dt", current_timestamp())
)

display(municipios_df)


In [0]:
# Arquivos de Qualificações de empresas
# Coluna dos arquivos de Qualificações de empresas
qualificacoes_columns = [
    "CÓDIGO",
    "DESCRICAO"
]

# Lendo e unificando os arquivos 
qualificacoes_df = spark.read.csv("dbfs:/tmp/deszipados/Qualificacoes*", header=False, encoding="iso-8859-1", sep=';')
qualificacoes_df = qualificacoes_df.toDF(*qualificacoes_columns)

# Criando uma coluna para gravar o caminho de origem dos dados
qualificacoes_df = qualificacoes_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Qualificacoes*"))

qualificacoes_df = (
    qualificacoes_df
    .withColumn("ingestion_dt", current_timestamp())    
)

display(qualificacoes_df)


In [0]:
# Arquivos de Naturezas
# Coluna dos arquivos de Naturezas
naturezas_columns = [
    "CÓDIGO",
    "DESCRICAO"
]

# Lendo e unificando os arquivos 
naturezas_df = spark.read.csv("dbfs:/tmp/deszipados/Naturezas*", header=False, encoding="iso-8859-1", sep=';')
naturezas_df = naturezas_df.toDF(*naturezas_columns)

# Criando uma coluna para gravar o caminho de origem dos dados
naturezas_df = naturezas_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Naturezas*"))


naturezas_df = (
    naturezas_df
    .withColumn("ingestion_dt", current_timestamp())
)

display(naturezas_df)


In [0]:
# Arquivos de Cnaes
# Coluna dos arquivos de Cnaes
cnaes_columns = [
    "CÓDIGO",
    "DESCRICAO"
]

# Lendo e unificando os arquivos 
cnaes_df = spark.read.csv("dbfs:/tmp/deszipados/Cnaes*", header=False, encoding="iso-8859-1", sep=';')
cnaes_df = cnaes_df.toDF(*cnaes_columns)

# Criando uma coluna para gravar o caminho de origem dos dados
cnaes_df = cnaes_df.withColumn("origin_path_name", lit("dbfs:/tmp/deszipados/Cnaes*"))

cnaes_df = (
    cnaes_df
    .withColumn("ingestion_dt", current_timestamp())
)

display(cnaes_df)
